In [1]:
#Imports
import pandas as pd
import numpy as np
from numpy import delete
import scipy.stats as stats

#Código necesario para correr funciones desde otro archivo .ipynb de la misma carpeta
%run Preparar_datos.ipynb
%run TablaResumenCC.ipynb

# Índice: Qué hay en el documento
 ¿Las oscilaciones diarias vienen del ramal?
- CCs sin suavizar de Tc-Te, Tc-Tr y Tr-Te
- CCs con los datos suavizados de Tc-Te, Tc-Tr y Tr-Te



Significado de las abreviaturas:

- CC: Coeficientes de correlación
- Tc: Temperatura del Cruce (ºC)
- Te: Temperatura del exterior (ºC)
- Tr: Temperatura del ramal del cruce (ºC)

Solo obtener la información para los mese con oscilaciones claras:
- 1) Los meses con oscilaciones claras para el C02 del cruce son: 
 Julio, agosto y septiembre de 2021.
 Junio, julio, agosto y septiembre de 2022
- 2) Los meses con oscilaciones claras para el CO2 del hall son:
Julio, agosto, septiembre y octubre de 2021. Mayo, junio y julio de 2022

# 1.1) ¿Las oscilaciones diarias vienen del ramal? Coeficientes a partir de las temperaturas sin suavizar

### Preparar datos:

In [2]:
#Obtener los datos por semanas Cruce:
#2021
SemanaCruceJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx")
SemanaCruceAgosto21 = SepararPorSemanas("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre21 = SepararPorSemanas("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")
SemanaCruceOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx")
#2022
SemanaCruceJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx")
SemanaCruceAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")

#Obtener los datos por semanas Exterior:
#2021
SemanaExteriorJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto21 = SepararPorSemanas( "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre21 = SepararPorSemanas( "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")
SemanaExteriorOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx")
#2022
SemanaExteriorJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")

### Obtener coeficientes:

In [3]:
def cc_Tr_Tc_Te(DataframeCruce, DataframeExterior):#Función adaptada para variables sin suavizar
    """
    Atributos:
    1) DataframeCruce: Lista de dataframes por semanas correspondiente a los datos de un mes de Cruce
    2) DataframeExterior: Lista de dataframes por semanas correspondiente a los datos de un mes de Exterior
    
    Devuelve:
    Una lista de listas, r. Cada elemento de r, r[i] con i=0, 1, 2, 3, 4, es una lista con los coeficiente de correlación y p 
    valores correspondientes a una semana.  r[i] = r_TD_Tr, p_TD_Tr, r_TD_Te, p_TD_Te, r_Tr_Te, p_Tr_Te
    
    ¿A qué corresponde cada elemento de r[i]?
    1) r_TD_Tr: Coeficiente de correlación entre la temperatura del Cruce y del Ramal
    2) p_TD_Tr: P-valor correspondiente a r_TD_Tr
    3) r_TD_Te: Coeficiente de correlación entre la temperatura del Cruce y la exterior
    4) p_TD_Te: P-valor correspondiente a r_TD_Te
    5) r_Tr_Te: Coeficiente de correlación entre la temperatura del Ramal y la exterior
    6) p_Tr_Te: P-valor correspondiente a r_Tr_Te
    """
    i=0
    r=[]
    while i<len(DataframeCruce):
       #print(i)
        DataframeCruce[i].reset_index(drop=True, inplace=True)
        DataframeExterior[i].reset_index(drop=True, inplace=True)
        
        # Variables que me van a hacer falta: Tc, Te, Tr
        for index, _ in DataframeExterior[i].iterrows():#Se va fuera de indice
            if  __builtins__.sum([pd.isna(DataframeCruce[i]["Temperatura D (° C)"][index]),
                                  pd.isna(DataframeCruce[i]['Tr_suavizada'][index]),
                                  pd.isna(DataframeExterior[i]["Tr Exterior (C Tr)"][index])]) > 0:
                DataframeCruce[i].drop([index], axis=0, inplace=True)
                DataframeExterior[i].drop([index], axis=0, inplace=True)

        #Mediante scipy por semanas:
        r_TD_Tr, p_TD_Tr = stats.pearsonr(DataframeCruce[i]["Temperatura D (° C)"], DataframeCruce[i]["tr Cruce Ramal ('C)"])
        r_TD_Te, p_TD_Te = stats.pearsonr(DataframeCruce[i]["Temperatura D (° C)"], DataframeExterior[i]["Tr Exterior (C Tr)"])
        r_Tr_Te, p_Tr_Te = stats.pearsonr(DataframeCruce[i]["tr Cruce Ramal ('C)"], DataframeExterior[i]["Tr Exterior (C Tr)"])
        rs = r_TD_Tr, p_TD_Tr, r_TD_Te, p_TD_Te, r_Tr_Te, p_Tr_Te
        r.append(rs)
        i=i+1
    return(r)

In [4]:
#Obtener las correlaciones entre la temperatura de cruce, ramal y exterior:
#2021
r7_2021 = cc_Tr_Tc_Te(SemanaCruceJulio21, SemanaExteriorJulio21)
r8_2021 = cc_Tr_Tc_Te(SemanaCruceAgosto21, SemanaExteriorAgosto21)
r9_2021 = cc_Tr_Tc_Te(SemanaCruceSeptiembre21, SemanaExteriorSeptiembre21)
r10_2021 = cc_Tr_Tc_Te(SemanaCruceOctubre21, SemanaExteriorOctubre21)
#2022
r7_2022 = cc_Tr_Tc_Te(SemanaCruceJulio22, SemanaExteriorJulio22)     
r8_2022 = cc_Tr_Tc_Te(SemanaCruceAgosto22, SemanaExteriorAgosto22)
r9_2022 = cc_Tr_Tc_Te(SemanaCruceSeptiembre22, SemanaExteriorSeptiembre22)

In [5]:
#Representar los valores de los coeficientes y p-valores en una tabla:
s1=pd.Series([r7_2021[0][0],r7_2021[1][0], r7_2021[2][0],r7_2021[3][0],r7_2021[4][0],
              r8_2021[0][0],r8_2021[1][0], r8_2021[2][0],r8_2021[3][0],r8_2021[4][0],#semanas de agosto 21
              r9_2021[0][0],r9_2021[1][0],r9_2021[2][0],r9_2021[3][0],r9_2021[4][0],#semanas de septiembre 21
              r10_2021[0][0],r10_2021[1][0],r10_2021[2][0],r10_2021[3][0],r10_2021[4][0],
              r7_2022[0][0], r7_2022[1][0], r7_2022[2][0], r7_2022[3][0], r7_2022[4][0],  
              r8_2022[0][0], r8_2022[1][0], r8_2022[2][0], r8_2022[3][0], r8_2022[4][0],
              r9_2022[0][0], r9_2022[1][0], r9_2022[2][0], r9_2022[3][0], r9_2022[4][0]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s2=pd.Series([r7_2021[0][1],r7_2021[1][1], r7_2021[2][1],r7_2021[3][1],r7_2021[4][1],
              r8_2021[0][1],r8_2021[1][1], r8_2021[2][1],r8_2021[3][1],r8_2021[4][1],#semanas de agosto 21
              r9_2021[0][1],r9_2021[1][1],r9_2021[2][1],r9_2021[3][1],r9_2021[4][1],#semanas de septiembre 21
              r10_2021[0][1],r10_2021[1][1],r10_2021[2][1],r10_2021[3][1],r10_2021[4][1],
              r7_2022[0][1], r7_2022[1][1], r7_2022[2][1], r7_2022[3][1], r7_2022[4][1],  
              r8_2022[0][1], r8_2022[1][1], r8_2022[2][1], r8_2022[3][1], r8_2022[4][1],
              r9_2022[0][1], r9_2022[1][1], r9_2022[2][1], r9_2022[3][1], r9_2022[4][1]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s3=pd.Series([r7_2021[0][2],r7_2021[1][2], r7_2021[2][2],r7_2021[3][2],r7_2021[4][2],
              r8_2021[0][2],r8_2021[1][2], r8_2021[2][2],r8_2021[3][2],r8_2021[4][2],#semanas de agosto 21
              r9_2021[0][2],r9_2021[1][2],r9_2021[2][2],r9_2021[3][2],r9_2021[4][2],#semanas de septiembre 21
              r10_2021[0][2],r10_2021[1][2],r10_2021[2][2],r10_2021[3][2],r10_2021[4][2],
              r7_2022[0][2], r7_2022[1][2], r7_2022[2][2], r7_2022[3][2], r7_2022[4][2], 
              r8_2022[0][2], r8_2022[1][2], r8_2022[2][2], r8_2022[3][2], r8_2022[4][2],
              r9_2022[0][2], r9_2022[1][2], r9_2022[2][2], r9_2022[3][2], r9_2022[4][2]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s4=pd.Series([r7_2021[0][3],r7_2021[1][3], r7_2021[2][3],r7_2021[3][3],r7_2021[4][3],
              r8_2021[0][3],r8_2021[1][3], r8_2021[2][3],r8_2021[3][3],r8_2021[4][3],#semanas de agosto 21
              r9_2021[0][3],r9_2021[1][3],r9_2021[2][3],r9_2021[3][3],r9_2021[4][3],
              r10_2021[0][3],r10_2021[1][3], r10_2021[2][3],r10_2021[3][3],r10_2021[4][3],
              r7_2022[0][3], r7_2022[1][3], r7_2022[2][3], r7_2022[3][3], r7_2022[4][3],
              r8_2022[0][3], r8_2022[1][3], r8_2022[2][3], r8_2022[3][3], r8_2022[4][3],
              r9_2022[0][3], r9_2022[1][3], r9_2022[2][3], r9_2022[3][3], r9_2022[4][3]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s5=pd.Series([r7_2021[0][4],r7_2021[1][4], r7_2021[2][4],r7_2021[3][4],r7_2021[4][4],
              r8_2021[0][4],r8_2021[1][4], r8_2021[2][4],r8_2021[3][4],r8_2021[4][4],#semanas de agosto 21
              r9_2021[0][4],r9_2021[1][4],r9_2021[2][4],r9_2021[3][4],r9_2021[4][4],#semanas de septiembre 21
              r10_2021[0][4],r10_2021[1][4],r10_2021[2][4],r10_2021[3][4],r10_2021[4][4],
              r7_2022[0][4], r7_2022[1][4], r7_2022[2][4], r7_2022[3][4], r7_2022[4][4], 
              r8_2022[0][4], r8_2022[1][4], r8_2022[2][4], r8_2022[3][4], r8_2022[4][4],
              r9_2022[0][4], r9_2022[1][4], r9_2022[2][4], r9_2022[3][4], r9_2022[4][4]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s6=pd.Series([r7_2021[0][5],r7_2021[1][5], r7_2021[2][5],r7_2021[3][5],r7_2021[4][5],
              r8_2021[0][5],r8_2021[1][5], r8_2021[2][5],r8_2021[3][5],r8_2021[4][5],#semanas de agosto 21
              r9_2021[0][5],r9_2021[1][5],r9_2021[2][5],r9_2021[3][5],r9_2021[4][5],#semanas de septiembre 21
              r10_2021[0][5],r10_2021[1][5],r10_2021[2][5],r10_2021[3][5],r10_2021[4][5],
              r7_2022[0][5], r7_2022[1][5], r7_2022[2][5], r7_2022[3][5], r7_2022[4][5],
              r8_2022[0][5], r8_2022[1][5], r8_2022[2][5], r8_2022[3][5], r8_2022[4][5],
              r9_2022[0][5], r9_2022[1][5], r9_2022[2][5], r9_2022[3][5], r9_2022[4][5]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])

#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['r_TD_Tr', 'p_TD_Tr', 'r_TD_Te', 'p_TD_Te', 'r_Tr_Te', 'p_Tr_Te']
display(CoefCorrel)
CoefCorrel.describe()

,r_TD_Tr,p_TD_Tr,r_TD_Te,p_TD_Te,r_Tr_Te,p_Tr_Te
Sem1 Julio 2021,0.031370,0.542630,-0.442141,1.432662e-19,-0.023955,0.642012
Sem2 Julio 2021,-0.000668,0.986203,-0.636375,1.425036e-77,0.005017,0.896709
Sem3 Julio 2021,-0.060015,0.120119,-0.502621,2.633099e-44,-0.001596,0.967053
Sem4 Julio 2021,-0.012018,0.755818,0.216960,1.334596e-08,0.059013,0.126443
Sem5 Julio 2021,0.039403,0.345172,-0.253735,6.499979e-10,0.032313,0.438913
Sem1 Agosto 2021,-0.084826,0.424030,-0.087882,4.074686e-01,0.139673,0.186681
Sem2 Agosto 2021,0.031269,0.418365,0.086891,2.428848e-02,-0.026226,0.497331
Sem3 Agosto 2021,0.049118,0.203486,0.444771,5.873126e-34,0.051356,0.183619
Sem4 Agosto 2021,0.044132,0.253266,0.659492,4.198765e-85,0.050363,0.192247
Sem5 Agosto 2021,0.082938,0.127500,0.620686,1.731705e-37,0.009223,0.865651


,r_TD_Tr,p_TD_Tr,r_TD_Te,p_TD_Te,r_Tr_Te,p_Tr_Te
count,35.000000,35.000000,35.000000,3.500000e+01,35.000000,35.000000
mean,0.032076,0.416537,0.338426,1.972210e-02,0.011067,0.465471
std,0.052491,0.288174,0.388170,7.909862e-02,0.067866,0.289791
min,-0.084826,0.000414,-0.636375,1.246242e-112,-0.090194,0.019360
25%,0.001751,0.188775,0.184552,7.125230e-78,-0.025090,0.194193
50%,0.031370,0.424030,0.444381,5.873126e-34,0.009223,0.460428
75%,0.058089,0.609088,0.649873,4.201294e-09,0.037928,0.700514
max,0.160539,0.986203,0.818776,4.074686e-01,0.291262,0.981229


### ¿Cuántos coeficientes hay en cada intervalo de valores?

In [6]:
ordenar_valores_cc (CoefCorrel, 'r_TD_Tr', 'p_TD_Tr', 'r_TD_Te', 'p_TD_Te', 'r_Tr_Te', 'p_Tr_Te')

,r_TD_Tr,r_TD_Te,r_Tr_Te
cc>0.5,0,16,0
cc>0,26,13,20
cc>-0.5,9,4,15
cc>-1,0,2,0
total,35,35,35


# 1.2) ¿Las oscilaciones diarias vienen del ramal? Coef a partir de las temperaturas suavizadas

### Preparar datos:

En el proceso de obtención de los coeficientes se borran datos. Es necesario volver a calcular las listas de dataframes por semanas cada vez que se quieran obtener coeficientes para evitar perder datos.

In [7]:
#Obtener los datos por semanas Cruce:
#2021
SemanaCruceJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx")
SemanaCruceAgosto21 = SepararPorSemanas("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre21 = SepararPorSemanas("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")
SemanaCruceOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx")
#2022
SemanaCruceJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx")
SemanaCruceAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")

#Obtener los datos por semanas Exterior:
#2021
SemanaExteriorJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto21 = SepararPorSemanas( "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre21 = SepararPorSemanas( "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")
SemanaExteriorOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx")
#2022
SemanaExteriorJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")

### Obtener coeficientes:

In [8]:
def cc_Tr_Tc_Te(DataframeCruce, DataframeExterior):#Función adaptada para variables suavizadas
    """
    Atributos:
    1) DataframeCruce: Lista de dataframes por semanas correspondiente a los datos de un mes de Cruce
    2) DataframeExterior: Lista de dataframes por semanas correspondiente a los datos de un mes de Exterior
    
    Devuelve:
    Una lista de listas, r. Cada elemento de r, r[i] con i=0, 1, 2, 3, 4, es una lista con los coeficiente de correlación y p 
    valores correspondientes a una semana.  r[i] = r_TD_Tr, p_TD_Tr, r_TD_Te, p_TD_Te, r_Tr_Te, p_Tr_Te
    
    ¿A qué corresponde cada elemento de r[i]?
    1) r_TD_Tr: Coeficiente de correlación entre la temperatura del Cruce y del Ramal
    2) p_TD_Tr: P-valor correspondiente a r_TD_Tr
    3) r_TD_Te: Coeficiente de correlación entre la temperatura del Cruce y la exterior
    4) p_TD_Te: P-valor correspondiente a r_TD_Te
    5) r_Tr_Te: Coeficiente de correlación entre la temperatura del Ramal y la exterior
    6) p_Tr_Te: P-valor correspondiente a r_Tr_Te
    """
    i=0
    r=[]
    while i<len(DataframeCruce):
       #print(i)
        DataframeCruce[i].reset_index(drop=True, inplace=True)
        DataframeExterior[i].reset_index(drop=True, inplace=True)
        
        # Variables que me van a hacer falta: Tc, Te, Tr
        for index, _ in DataframeExterior[i].iterrows():#Se va fuera de indice
            if  __builtins__.sum([pd.isna(DataframeCruce[i]['Tc_suavizada'][index]),
                                  pd.isna(DataframeCruce[i]['Tr_suavizada'][index]),
                                  pd.isna(DataframeExterior[i]['Te_suavizada'][index])]) > 0:
                DataframeCruce[i].drop([index], axis=0, inplace=True)
                DataframeExterior[i].drop([index], axis=0, inplace=True)

        #Mediante scipy por semanas:
        r_TD_Tr, p_TD_Tr = stats.pearsonr(DataframeCruce[i]['Tc_suavizada'], DataframeCruce[i]['Tr_suavizada'])
        r_TD_Te, p_TD_Te = stats.pearsonr(DataframeCruce[i]['Tc_suavizada'], DataframeExterior[i]['Te_suavizada'])
        r_Tr_Te, p_Tr_Te = stats.pearsonr(DataframeCruce[i]['Tr_suavizada'], DataframeExterior[i]['Te_suavizada'])
        rs = r_TD_Tr, p_TD_Tr, r_TD_Te, p_TD_Te, r_Tr_Te, p_Tr_Te
        r.append(rs)
        i=i+1
    return(r)

In [9]:
#Obtener las correlaciones entre la temperatura de cruce, ramal y exterior:
#2021
r7_2021 = cc_Tr_Tc_Te(SemanaCruceJulio21, SemanaExteriorJulio21)
r8_2021 = cc_Tr_Tc_Te(SemanaCruceAgosto21, SemanaExteriorAgosto21)
r9_2021 = cc_Tr_Tc_Te(SemanaCruceSeptiembre21, SemanaExteriorSeptiembre21)
r10_2021 = cc_Tr_Tc_Te(SemanaCruceOctubre21, SemanaExteriorOctubre21)
#2022
r7_2022 = cc_Tr_Tc_Te(SemanaCruceJulio22, SemanaExteriorJulio22)     
r8_2022 = cc_Tr_Tc_Te(SemanaCruceAgosto22, SemanaExteriorAgosto22)
r9_2022 = cc_Tr_Tc_Te(SemanaCruceSeptiembre22, SemanaExteriorSeptiembre22)

In [10]:
#Representar los valores de los coeficientes y p-valores en una tabla:
s1=pd.Series([r7_2021[0][0],r7_2021[1][0], r7_2021[2][0],r7_2021[3][0],r7_2021[4][0],
              r8_2021[0][0],r8_2021[1][0], r8_2021[2][0],r8_2021[3][0],r8_2021[4][0],#semanas de agosto 21
              r9_2021[0][0],r9_2021[1][0],r9_2021[2][0],r9_2021[3][0],r9_2021[4][0],#semanas de septiembre 21
              r10_2021[0][0],r10_2021[1][0],r10_2021[2][0],r10_2021[3][0],r10_2021[4][0],
              r7_2022[0][0], r7_2022[1][0], r7_2022[2][0], r7_2022[3][0], r7_2022[4][0],  
              r8_2022[0][0], r8_2022[1][0], r8_2022[2][0], r8_2022[3][0], r8_2022[4][0],
              r9_2022[0][0], r9_2022[1][0], r9_2022[2][0], r9_2022[3][0], r9_2022[4][0]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s2=pd.Series([r7_2021[0][1],r7_2021[1][1], r7_2021[2][1],r7_2021[3][1],r7_2021[4][1],
              r8_2021[0][1],r8_2021[1][1], r8_2021[2][1],r8_2021[3][1],r8_2021[4][1],#semanas de agosto 21
              r9_2021[0][1],r9_2021[1][1],r9_2021[2][1],r9_2021[3][1],r9_2021[4][1],#semanas de septiembre 21
              r10_2021[0][1],r10_2021[1][1],r10_2021[2][1],r10_2021[3][1],r10_2021[4][1],
              r7_2022[0][1], r7_2022[1][1], r7_2022[2][1], r7_2022[3][1], r7_2022[4][1],  
              r8_2022[0][1], r8_2022[1][1], r8_2022[2][1], r8_2022[3][1], r8_2022[4][1],
              r9_2022[0][1], r9_2022[1][1], r9_2022[2][1], r9_2022[3][1], r9_2022[4][1]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s3=pd.Series([r7_2021[0][2],r7_2021[1][2], r7_2021[2][2],r7_2021[3][2],r7_2021[4][2],
              r8_2021[0][2],r8_2021[1][2], r8_2021[2][2],r8_2021[3][2],r8_2021[4][2],#semanas de agosto 21
              r9_2021[0][2],r9_2021[1][2],r9_2021[2][2],r9_2021[3][2],r9_2021[4][2],#semanas de septiembre 21
              r10_2021[0][2],r10_2021[1][2],r10_2021[2][2],r10_2021[3][2],r10_2021[4][2],
              r7_2022[0][2], r7_2022[1][2], r7_2022[2][2], r7_2022[3][2], r7_2022[4][2], 
              r8_2022[0][2], r8_2022[1][2], r8_2022[2][2], r8_2022[3][2], r8_2022[4][2],
              r9_2022[0][2], r9_2022[1][2], r9_2022[2][2], r9_2022[3][2], r9_2022[4][2]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s4=pd.Series([r7_2021[0][3],r7_2021[1][3], r7_2021[2][3],r7_2021[3][3],r7_2021[4][3],
              r8_2021[0][3],r8_2021[1][3], r8_2021[2][3],r8_2021[3][3],r8_2021[4][3],#semanas de agosto 21
              r9_2021[0][3],r9_2021[1][3],r9_2021[2][3],r9_2021[3][3],r9_2021[4][3],
              r10_2021[0][3],r10_2021[1][3], r10_2021[2][3],r10_2021[3][3],r10_2021[4][3],
              r7_2022[0][3], r7_2022[1][3], r7_2022[2][3], r7_2022[3][3], r7_2022[4][3],
              r8_2022[0][3], r8_2022[1][3], r8_2022[2][3], r8_2022[3][3], r8_2022[4][3],
              r9_2022[0][3], r9_2022[1][3], r9_2022[2][3], r9_2022[3][3], r9_2022[4][3]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s5=pd.Series([r7_2021[0][4],r7_2021[1][4], r7_2021[2][4],r7_2021[3][4],r7_2021[4][4],
              r8_2021[0][4],r8_2021[1][4], r8_2021[2][4],r8_2021[3][4],r8_2021[4][4],#semanas de agosto 21
              r9_2021[0][4],r9_2021[1][4],r9_2021[2][4],r9_2021[3][4],r9_2021[4][4],#semanas de septiembre 21
              r10_2021[0][4],r10_2021[1][4],r10_2021[2][4],r10_2021[3][4],r10_2021[4][4],
              r7_2022[0][4], r7_2022[1][4], r7_2022[2][4], r7_2022[3][4], r7_2022[4][4], 
              r8_2022[0][4], r8_2022[1][4], r8_2022[2][4], r8_2022[3][4], r8_2022[4][4],
              r9_2022[0][4], r9_2022[1][4], r9_2022[2][4], r9_2022[3][4], r9_2022[4][4]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s6=pd.Series([r7_2021[0][5],r7_2021[1][5], r7_2021[2][5],r7_2021[3][5],r7_2021[4][5],
              r8_2021[0][5],r8_2021[1][5], r8_2021[2][5],r8_2021[3][5],r8_2021[4][5],#semanas de agosto 21
              r9_2021[0][5],r9_2021[1][5],r9_2021[2][5],r9_2021[3][5],r9_2021[4][5],#semanas de septiembre 21
              r10_2021[0][5],r10_2021[1][5],r10_2021[2][5],r10_2021[3][5],r10_2021[4][5],
              r7_2022[0][5], r7_2022[1][5], r7_2022[2][5], r7_2022[3][5], r7_2022[4][5],
              r8_2022[0][5], r8_2022[1][5], r8_2022[2][5], r8_2022[3][5], r8_2022[4][5],
              r9_2022[0][5], r9_2022[1][5], r9_2022[2][5], r9_2022[3][5], r9_2022[4][5]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])

#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['r_TD_Tr', 'p_TD_Tr', 'r_TD_Te', 'p_TD_Te', 'r_Tr_Te', 'p_Tr_Te']
display(CoefCorrel)
CoefCorrel.describe()

,r_TD_Tr,p_TD_Tr,r_TD_Te,p_TD_Te,r_Tr_Te,p_Tr_Te
Sem1 Julio 2021,-0.000601,9.906942e-01,-0.546704,6.372187e-31,-0.029920,0.561449
Sem2 Julio 2021,-0.016933,6.612574e-01,-0.709943,4.145792e-104,0.014672,0.704194
Sem3 Julio 2021,-0.073068,5.833773e-02,-0.570587,2.661395e-59,-0.000420,0.991334
Sem4 Julio 2021,-0.014819,7.013775e-01,0.265943,2.421144e-12,0.131249,0.000648
Sem5 Julio 2021,0.059375,1.546865e-01,-0.311850,1.859748e-14,0.086603,0.037721
Sem1 Agosto 2021,-0.309065,2.874262e-03,-0.103446,3.291690e-01,0.280567,0.007065
Sem2 Agosto 2021,0.107098,5.450584e-03,0.104935,6.475287e-03,-0.057430,0.136961
Sem3 Agosto 2021,0.142998,1.998867e-04,0.485979,4.008029e-41,0.125145,0.001150
Sem4 Agosto 2021,0.115220,2.778454e-03,0.695604,2.637864e-98,0.124358,0.001237
Sem5 Agosto 2021,0.165187,2.279587e-03,0.708978,4.715003e-53,0.010678,0.844704


,r_TD_Tr,p_TD_Tr,r_TD_Te,p_TD_Te,r_Tr_Te,p_Tr_Te
count,35.000000,3.500000e+01,35.000000,3.500000e+01,35.000000,35.000000
mean,0.052386,1.930583e-01,0.344075,9.734274e-03,0.026789,0.253487
std,0.129697,2.664986e-01,0.453372,5.559857e-02,0.154355,0.326240
min,-0.309065,5.167104e-16,-0.759211,2.224198e-136,-0.199502,0.000003
25%,-0.011478,2.747658e-03,0.199202,1.225007e-91,-0.056502,0.005438
50%,0.063777,1.003352e-01,0.485979,4.008029e-41,0.014240,0.074946
75%,0.120124,2.377629e-01,0.698729,2.881932e-11,0.088794,0.545132
max,0.358611,9.906942e-01,0.848187,3.291690e-01,0.695988,0.991334


### ¿Cuántos coeficientes hay en cada intervalo de valores?

In [11]:
ordenar_valores_cc (CoefCorrel, 'r_TD_Tr', 'p_TD_Tr', 'r_TD_Te', 'p_TD_Te', 'r_Tr_Te', 'p_Tr_Te')

,r_TD_Tr,r_TD_Te,r_Tr_Te
cc>0.5,0,17,1
cc>0,24,12,18
cc>-0.5,11,2,16
cc>-1,0,4,0
total,35,35,35
